In [ ]:
import numpy as np
from lib import *
from plotly import graph_objects as go 
from scipy.special import wofz
from scipy.signal.windows import kaiser
import lmfit

# This notebook tests the basic numerical functions in the library

## Test analytical functions of FT

### FT of FID: exp(-at^2-bt)

Compares the analytical expression of the FT of the FID function, with the numerical FT of the FID function.

In [ ]:
tlen = 2**12; tzp = tlen * 4     # length of time array & zero-padding array
trunc = 10   # time array truncation factor: truncate to intensity = exp(-10)

fig_res = go.Figure()    # residual figure
for a, b in [(1, -2), (1, 0), (1, 2), (0, 0.1)]:
    # find time array truncation to avoid spectral leakage
    if a > 0:
        tmax = (np.sqrt(b**2 + 4 * a * trunc) - b) / (2 * a)
    else:
        tmax = trunc / b 
    t = np.linspace(0, tmax, num=tlen)
    xx = np.fft.fftshift(np.fft.fftfreq(tzp)) / tmax * tlen   # x array in freq domain

    # numerical FFT 
    fid = np.exp(-a * t**2 - b * t)
    yy = np.fft.fftshift(np.abs(np.fft.fft(fid, tzp)))
    
    # analytical solution using time domain integral
    yat = fid_ft_ana(xx, a, b, yshift=0)
    
    # make figure
    fig = go.Figure(layout=go.Layout(title='a={:g}, b={:g}'.format(a, b)))
    fig.update_xaxes(range=[-10, 10])
    fig.add_trace(go.Scatter(x=xx, y=yy/yy.max(), name='Numerical FFT'))
    fig.add_trace(go.Scatter(x=xx, y=yat/yat.max(), name='Analytical Expression'))
    fig.show()
    fig_res.add_trace(go.Scatter(x=xx, y=yy/yy.max()-yat/yat.max(), name='YAT a={:g}, b={:g}'.format(a, b)))
fig_res.update_layout(title='Residuals')
fig_res.show()

### FT of Voigt-1D window: t * exp(-at^2-bt)

In [ ]:
tlen = 2**12; tzp = tlen * 4     # length of time array & zero-padding array
trunc = 10   # time array truncation factor: truncate to intensity = exp(-10)

fig_res = go.Figure()    # residual figure
for a, b in [(1, -2), (1, 0), (1, 2), (0, 0.1)]:
    # find time array truncation to avoid spectral leakage
    if a > 0:
        tmax = (np.sqrt(b**2 + 4 * a * trunc) - b) / (2 * a)
    else:
        tmax = trunc / b + np.log(trunc / b) / b  # extra length for the time array
    t = np.linspace(0, tmax, num=tlen)
    xx = np.fft.fftshift(np.fft.fftfreq(tzp)) / tmax * tlen   # x array in freq domain

    # numerical FFT 
    fid = np.exp(-a * t**2 - b * t) * t
    yy = np.fft.fftshift(np.abs(np.fft.fft(fid, tzp)))
    
    # analytical solution using time domain integral
    yat = voigt1d_ft_ana(xx, a, b, yshift=0)
    
    # make figure
    fig = go.Figure(layout=go.Layout(title='a={:g}, b={:g}'.format(a, b)))
    fig.update_xaxes(range=[-10, 10])
    fig.add_trace(go.Scatter(x=xx, y=yy/yy.max(), name='Numerical FFT'))
    fig.add_trace(go.Scatter(x=xx, y=yat/yat.max(), name='Analytical Expression'))
    fig.show()
    fig_res.add_trace(go.Scatter(x=xx, y=yy/yy.max()-yat/yat.max(), name='YAT a={:g}, b={:g}'.format(a, b)))
fig_res.update_layout(title='Residuals')
fig_res.show()

### Parameter conversion

Test the parameter conversion for Complex Voigt profile:
* $a = \pi^2\gamma_G^2 / (4\ln 2) \leftrightarrow \gamma_G = 2\sqrt{a\ln 2} / \pi$ 
* $b = \pi \gamma_L \leftrightarrow \gamma_L = b / \pi$

In [ ]:
for i in range(4):
    gg = abs(np.random.rand())
    ll = abs(np.random.rand())
    a = (np.pi * gg)**2 / (4 * np.log(2))
    b = np.pi * ll
    tlen = 2**12; tzp = tlen * 4     # length of time array & zero-padding array
    trunc = 10   # time array truncation factor: truncate to intensity = exp(-10)

    # find time array truncation to avoid spectral leakage
    if a > 0:
        tmax = (np.sqrt(b**2 + 4 * a * trunc) - b) / (2 * a)
    else:
        tmax = trunc / b 
    t = np.linspace(0, tmax, num=tlen)
    xx = np.fft.fftshift(np.fft.fftfreq(tzp)) / tmax * tlen   # x array in freq domain

    # numerical FFT 
    tfid = np.exp(-a * t**2 - b * t)
    tv1d = t * tfid
    ft_fid = np.fft.fftshift(np.abs(np.fft.fft(tfid, tzp)))   # 1st
    ft_v1d = np.fft.fftshift(np.abs(np.fft.fft(tv1d, tzp)))   # 2nd

    # voigt profile
    yvc = complex_voigt(xx, gg, ll)
    yv = voigt(xx, gg, ll)
    ygau = np.abs(2 * np.pi * xx / np.sqrt(a) * wofz(-np.pi * xx / np.sqrt(a)) + 2 * 1j / np.sqrt(np.pi))

    fig1 = go.Figure(layout=go.Layout(title='a={:.4f}, b={:.4f}, gg={:.4f}, ll={:.4f}'.format(a, b, gg, ll)))
    fig1.update_xaxes(range=[-10, 10])
    
    fig1.add_trace(go.Scatter(x=xx, y=ft_fid/ft_fid.max(), name='FFT'))
    fig1.add_trace(go.Scatter(x=xx, y=yvc, name='Complex Voigt'))
    fig1.show()

### Test numericall full width finder

Validate the result range of the finder. The `fw_num` (interpolated) should be always between `min` and `max`:
the distance between the $x$ points just above or below $y_\text{max}/2$. 

In [ ]:
x = np.linspace(0, 5, num=101)
y = np.exp(-x**2)
print('{:^6s} {:^6s} {:^6s} {:^6s}'.format('min', 'fw_num', 'max', 'min<fw<max?'))
for i in range(10):
    xx = np.fft.fftshift(np.fft.fftfreq(101*2**i)) / 5 * 101
    yy = np.fft.fftshift(np.abs(np.fft.fft(y, 101*2**i)))
    vv = fwhm_num(xx, yy)
    vv_min = xx[yy >= 0.5 * yy.max()].ptp()
    xx_less = xx[yy < 0.5 * yy.max()]
    vv_max = xx_less[xx_less > 0].min() - xx_less[xx_less < 0].max()
    print('{:>6.4f} {:>6.4f} {:>6.4f} {:^6d}'.format(vv_min, vv, vv_max, vv_min < vv < vv_max))


### Test convergence of zero-padding level for numerical full width

Zero-padding length affects the finesse of the x grid on FT. This could affect the accuracy of numerical FW determination. Therefore, we need to test how much zero-padding is sufficient to get accurate fw.

#### For Kaiser window

In [ ]:
a0 = 1.0; b0 = 0.0
x = np.linspace(0, 1, num=2**10)
y = np.exp(-a0*x**2-b0*x) 
fig = go.Figure()

for pi_alpha in range(0, 17, 4):
    yk = y * kaiser(len(x), pi_alpha)
    vv = []
    for i in range(11, 24):
        xx = np.fft.fftshift(np.fft.fftfreq(2**i)) / 1 * 2**10
        yy = np.fft.fftshift(np.abs(np.fft.fft(yk, 2**i)))
        vv.append(fwhm_num(xx, yy))
    fig.add_trace(go.Scatter(x=list(range(11, 24)), y=vv, name=str(pi_alpha)))

fig.update_layout(
    title="a0 = 1.0, b0 = 0.0",
    xaxis_title="FFT zero-padding length (log2 scale)",
    yaxis_title="Numerical FWHM",
    legend_title="πα"
)

fig.show()


This test shows that a zero-padding factor of $2^6$ is sufficient to get a reliable estimation of the numerical FWHM for Kaiser-Bessel window up to 4 decimal points.

#### For Voigt-1D window

In [ ]:
fig = go.Figure()
b_grid = np.arange(-5, 6)
trunc = 10

for a in [0, 1, 5]:
    fig = go.Figure()
    for b in b_grid:
        if a == 0 and b <= 0:   # skip invalid parameters
            pass
        else:
            if a > 0:
                tmax = (np.sqrt(b**2 + 4 * a * trunc) - b) / (2 * a)
            else:
                tmax = trunc / b +  np.log(trunc / b) / b  # extra length for the time array
            
            t = np.linspace(0, tmax, num=2**10)
            y = np.exp(-a*t**2-b*t)*t 
            vv = []
            for i in range(11, 24):
                xx = np.fft.fftshift(np.fft.fftfreq(2**i)) / 1 * 2**10
                yy = np.fft.fftshift(np.abs(np.fft.fft(y, 2**i)))
                vv.append(fwhm_num(xx, yy))
        fig.add_trace(go.Scatter(x=list(range(11, 24)), y=vv, name=str(b)))
    fig.update_layout(
        title="a = {:d}".format(a),
        xaxis_title="FFT zero-padding length (log2 scale)",
        yaxis_title="Numerical FWHM",
        legend_title="b"
    )
    fig.show()


This test shows that a zero-padding factor of $2^6$ is sufficient to get a reliable estimation of the numerical FWHM for Voigt-1D window up to 4 decimal points.

#### Test precision of Voigt-1D numerical FWHM solvers: quadratic fitting v.s. root-finding

The test result shows that two approaches return identical results.

In [ ]:
x = np.linspace(-100, 100, 10**5+1)
vv_quad = []
vv_root = []
for a in np.arange(0, 5.1, 0.5):
    for b in np.arange(-5, 5.1, 0.5):
        if b <= 0 and abs(b) >= 2*np.sqrt(a):
            pass
        else:
            y = voigt1d_ft_ana(x, a, b, yshift=0)
            vv_root.append(fwhm_ab(a, b))
            vv_quad.append(fwhm_num(x, y))
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(vv_quad)), y=vv_quad, name='quad', mode='markers'))
fig.add_trace(go.Scatter(x=np.arange(len(vv_quad)), y=vv_root, name='root', mode='markers'))
fig.show()